# Automated ML

## Overview

For this notebook we will be importing the dataset, creating an experiment, creating a compute cluster, cleanning and preprocessing the data before doing our train and test split, running the AutoML Experiment and selecting the best AutoML Model.

## Dependencies

First we will be importing all the needed dependencies to complete the project.

Warning: Install the `imblearn` library before running the rest of the notebook

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from azureml.core import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.automl import AutoMLConfig

from azureml.pipeline.steps import AutoMLStep

from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


## Workspace Configuration

In this cell we import the workspace configuration and create an experiment that we will use later.

In [2]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'automlcovid'

experiment=Experiment(ws, experiment_name)
experiment

Workspace name: quick-starts-ws-144339
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-144339


Name,Workspace,Report Page,Docs Page
automlcovid,quick-starts-ws-144339,Link to Azure Machine Learning studio,Link to Documentation


## Compute Cluster creation
In this cell a cpu cluster is created for running our experiments, it checks if a compute cluster with the same name exists, if it exists then uses it, if not it creates it.

If the cluster does not exists we define the configuration for it. For this project we will be using `min_nodes = 1`, in your own project this will incurr in extra costs, so consider leaving it to 0.

In [3]:
compute_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing compute cluster...")
except:
    print("Creating new compute cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16s_V3', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print("Cluster details: ", compute_target.get_status().serialize())

Found existing compute cluster...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Cluster details:  {'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 4, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-10T03:36:42.954000+00:00', 'errors': None, 'creationTime': '2021-05-10T01:43:20.670500+00:00', 'modifiedTime': '2021-05-10T01:43:36.013311+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D16S_V3'}


## Dataset

### Overview
We are going to use the Mexican Government's COVID-19 data, once it is uploaded to ML Studio we will consume it using TabularDataset.

To consume the dataset we will import it from the Datastore tab in ML Studio, this is a blob storage, then we have to specify the location of the csv to import it.

Once it's imported we will we creating a Pandas Dataframe that we will need later on.

In [4]:
datastore = Datastore.get(ws,'workspaceblobstore')
ds = TabularDatasetFactory.from_delimited_files(path=(datastore, 'UI/05-10-2021_013755_UTC/210509COVID19MEXICO.csv'))
df = ds.to_pandas_dataframe()

print(df)

        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
0                2021-05-09      z482b8       1      12           9     2   
1                2021-05-09      z49a69       1      12          23     1   
2                2021-05-09      z23d9d       1      12          22     2   
3                2021-05-09      z24953       1      12           9     1   
4                2021-05-09      zz8e77       1      12           9     2   
...                     ...         ...     ...     ...         ...   ...   
6757355          2021-05-09      6bd21b       2      12           9     1   
6757356          2021-05-09      d0d932       2      12          12     1   
6757357          2021-05-09      7ceaf8       2       4           9     1   
6757358          2021-05-09      56cd1e       2       4           5     1   
6757359          2021-05-09      cd3ffd       2       4          15     2   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
0  

### Data Preprocessing

#### Cleaning
The objective of our project is to create a model that can get if a patient with COVID-19 will enter into an Intensive Care Unit or not.

The objective variable `UCI` has a lot of values that are 97, 98, or 99, which means that we don't know if the person is or was in a Intensive Care Unit, we only need the values 1 and 2, which means that the patient was or wasn't in an Intensive Care Unit, respectively.

So the next step is to trim our dataset by removing the rows that have 97, 98 or 99, leaving only 'Yes'/'No' for our objective variable.

In [5]:
arr_drop = [97,98,99]
for element in arr_drop:
    df.drop(df.loc[df['UCI']==element].index,inplace=True)

print(df)
print("--------------------------------------")
print("Classes count")
classes = pd.crosstab(index=df['UCI'],columns="count")
print(classes)


        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
1                2021-05-09      z49a69       1      12          23     1   
16               2021-05-09      z166d5       1      12           1     1   
21               2021-05-09      z388cd       1      12          29     2   
22               2021-05-09      z4533d       1      12          29     1   
27               2021-05-09      z4494e       1      12           8     1   
...                     ...         ...     ...     ...         ...   ...   
6757282          2021-05-09      4b89bb       1      12          22     2   
6757292          2021-05-09      7efdce       1       4           9     2   
6757308          2021-05-09      d45a16       1       4          19     1   
6757320          2021-05-09      d3d30f       2       6          28     1   
6757347          2021-05-09      a7f5d8       1      12          18     1   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
1  

### Defining useful features
As we defined in our task definition in the `README.md` file, we will use only the following features: `SEXO`, `EDAD`, `NEUMONIA`, `DIABETES`, `EPOC`, `ASMA`, `INMUSUPR`, `HIPERTENSION`, `OTRA_COM`, `CARDIOVASCULAR`, `OBESIDAD`, `RENAL_CRONICA`, `TABAQUISMO`, `RESULTADO_LAB`.

In [6]:
df = df[['SEXO', 'EDAD','NEUMONIA','DIABETES','EPOC','ASMA','INMUSUPR','HIPERTENSION','OTRA_COM','CARDIOVASCULAR','OBESIDAD','RENAL_CRONICA','TABAQUISMO','RESULTADO_LAB', 'UCI']]
print(df)

         SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
1           1    66         1         1     2     2         2             1   
16          1    32         1         1     2     2         2             1   
21          2    12         1         1     2     2         2             2   
22          1    11         2         1     2     2         2             2   
27          1    49         2         2     2     2         2             2   
...       ...   ...       ...       ...   ...   ...       ...           ...   
6757282     2    87         1         2     2     2         2             2   
6757292     2    21         2         2     2     2         2             2   
6757308     1    69         2         2     2     2         2             2   
6757320     1    70         2         2     1     2         1             1   
6757347     1    33         1         2     2     2         2             2   

         OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_

#### Handling dataset class imbalance

As we can see, the rows with 1 (Yes) in the `UCI` column are a lot less than the 2 (No). we can send the dataset to training like this, but we will get a "Class balancing warning" alert in Azure ML, so we are going to fix it.

The next step in our data preprocessing is to balance the dataset. We will need the `imblearn` library that we installed before, and we will need to divide our dataframe in X and y to use the "RandomUnderSampler" method.

`y` has only the column `UCI` column, and `X` the rest of the columns, as we already defined the features we wanted in the previous cell

In [7]:
y = df['UCI']
X = df.drop('UCI',1)

rus = RandomUnderSampler(random_state = 42)
X_resampled, y_resampled = rus.fit_resample(X,y)
print(X_resampled)
print("-------")
print(y_resampled)

        SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
0          1    66         1         1     2     2         2             1   
1          2    60         1         1     2     2         2             2   
2          1    80         1         1     2     2         2             1   
3          1     5         1         2     2     2         2             2   
4          1    58         1         2     2     2         2             2   
...      ...   ...       ...       ...   ...   ...       ...           ...   
109779     2    15         2         2     2     2         2             2   
109780     2    86         2         2     2     2         2             1   
109781     2    61         1         1     2     2         2             1   
109782     2    63         1         2     2     2         2             1   
109783     1    49         1         2     2     2         2             2   

        OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  TABA

### Train/test split

Now that we have our balanced data, we will divide X and y between train and test

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled,y_resampled,test_size = 0.2, random_state = 42)

train_data = pd.concat([x_train,y_train],axis=1)

print(train_data)

print("Validating that we have balanced data for training.")
classes = pd.crosstab(index=train_data['UCI'],columns="count")
print(classes)

        SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
29634      1    23         1         2     2     2         2             2   
106415     2    74         1         1     2     2         2             1   
78580      1    71         2         1     2     2         2             1   
45877      2    34         1         2     2     2         2             2   
524        2    33         1         2     2     2         2             2   
...      ...   ...       ...       ...   ...   ...       ...           ...   
54886      1    49         2         2     2     2         2             1   
76820      2    63         2         2     2     2         2             2   
103694     2    56         2         1     2     2         2             1   
860        2    46         1         2     2     2         2             2   
15795      2    32         2         2     2     2         2             2   

        OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  TABA

### Converting Train Data to Tabular Dataset
Since the train data is in the pandas Dataframe format, we have to convert it to Tabular Dataset to be used in AutoML

Warning: don't forget to drop the dataframe index, as the csv creates another index, and it will confuse our model.

In [12]:
#Convert train_data (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path = 'data/traindata.csv'
train_data.to_csv(path, index= False)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/traindata.csv'))])
print("Successfully converted the dataset to TabularDataset format.")
print(type(train_data))

Uploading an estimated of 5 files
Target already exists. Skipping upload for data/train.csv
Target already exists. Skipping upload for data/train2.csv
Target already exists. Skipping upload for data/train3.csv
Target already exists. Skipping upload for data/trainBalanced.csv
Uploading data/traindata.csv
Uploaded data/traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Successfully converted the dataset to TabularDataset format.
<class 'azureml.data.tabular_dataset.TabularDataset'>


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Since I was dealing with a large dataset, I needed to give more time to the experiment_timeout_minutes, that's why it's set to 1 hour.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "classification",
    training_data=train_data,
    label_column_name='UCI',   
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

## Submit the experiment with RunDetails widget
With the `RunDetails` widget we can see important outputs and when the run is completed.

In this cell we submit the experiment, with `show_output = True` to see the run logs in real time

In [17]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
#Submit the experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automlcovid,AutoML_de361d0a-2f6f-4000-90b9-4cd588c5c22f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage                       |
+=================================+=================================+=================================+
|train                            |79044          

## Best Model

In the cell below we get the best model from the automl experiments and display all the properties of the model.



In [21]:
best_run_automl, fitted_model_automl = remote_run.get_output()

#Displaying properties of the AutoML Model
print("AutoML:")
automl_best_run_metrics = best_run_automl.get_metrics()
print("Best run Id: ",best_run_automl.id)
print("Accuracy: ", automl_best_run_metrics['accuracy'])
print("All metrics:")
for metric in automl_best_run_metrics:
    value = automl_best_run_metrics[metric]
    print(metric,value)
#Saving the best model
model_automl = best_run_automl.register_model(model_path='./outputs/', model_name='model_automl.pkl')
print("\nModel saved successfully")

AutoML:
Best run Id:  AutoML_de361d0a-2f6f-4000-90b9-4cd588c5c22f_38
Accuracy:  0.6420357508823864
All metrics:
AUC_weighted 0.684852452885799
f1_score_weighted 0.6351655361242334
log_loss 0.6358452360325219
AUC_micro 0.6915708695789867
average_precision_score_macro 0.673512427213038
average_precision_score_micro 0.6808832612149752
weighted_accuracy 0.6417067693627544
balanced_accuracy 0.6423647361634816
accuracy 0.6420357508823864
norm_macro_recall 0.2847294723269631
average_precision_score_weighted 0.6735466258642014
precision_score_weighted 0.6540698467413734
f1_score_micro 0.6420357508823864
recall_score_micro 0.6420357508823864
AUC_macro 0.6848524528857989
precision_score_micro 0.6420357508823864
matthews_correlation 0.2961034924403513
f1_score_macro 0.6352841829251554
precision_score_macro 0.6539659339070646
recall_score_weighted 0.6420357508823864
recall_score_macro 0.6423647361634816
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_de361d0a-2f6f-4000-90b9-4cd588c5c22f_

In [ ]:
# Displaying the fitted model properties
print_model(fitted_model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Before this step we have to run the `hyperparameter_tuning.ipynb`  notebook, in this case we determined that the model with the most accuracy was the one from AutoML. The next step is to deploy the model and test it.

First we have to download the scoring file and the environment file, this can be obtained from the best run.

In [25]:
# Download scoring file 
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

Now we have to deploy the model with inference configuration

In [27]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

#Configuration for inference
inference_config = InferenceConfig(entry_script='scoring.py',
                                    environment=best_run_automl.get_environment())

#Deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = model_automl.deploy(ws, "automlcovid", [model_automl], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print("Service state:")
print(service.state)
print("Scoring URI:")
print(service.scoring_uri)
print("Swagger URI:")
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-10 06:07:27+00:00 Creating Container Registry if not exists..
2021-05-10 06:07:38+00:00 Registering the environment..
2021-05-10 06:07:39+00:00 Use the existing image.
2021-05-10 06:07:39+00:00 Generating deployment configuration.
2021-05-10 06:07:41+00:00 Submitting deployment to compute..
2021-05-10 06:07:44+00:00 Checking the status of deployment automlcovid..
2021-05-10 06:10:59+00:00 Checking the status of inference endpoint automlcovid.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 060a8f2c-2942-456c-a6ed-00c6f6ac6be6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/model_automl.pkl/6/model.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/model_automl.pkl/6/model.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 060a8f2c-2942-456c-a6ed-00c6f6ac6be6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/model_automl.pkl/6/model.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/model_automl.pkl/6/model.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 060a8

In the cell below we send a request to the web service we deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service